# Assignment 7

**Course**: AM5801 - Computational Lab  
**Name**: Atharv Shete  
**Roll No**: BE22B021

## Question 1
Running Dijkstra's algorithm from vertex C to determine the next vertex visited.

In [15]:
from heapq import heappush, heappop

INF = float("inf")


ASSIGNMENT_GRAPH = {
    "A": {"C": 3, "D": 4, "E": 4},
    "B": {"C": 2, "F": 2},
    "C": {"A": 3, "B": 2, "E": 4, "F": 5, "G": 5},
    "D": {"A": 4, "E": 2},
    "E": {"A": 4, "C": 4, "D": 2, "G": 5},
    "F": {"B": 2, "C": 5, "G": 5},
    "G": {"C": 5, "E": 5, "F": 5},
}


def dijkstra_shortest_paths(graph, start):
    """Run Dijkstra's algorithm from `start` and capture distances and traversal order."""
    if start not in graph:
        raise ValueError(f"Start vertex '{start}' is not present in the graph.")

    distances = {vertex: INF for vertex in graph}
    predecessor = {vertex: None for vertex in graph}
    visit_order = []

    distances[start] = 0.0
    frontier = [(0.0, start)]

    while frontier:
        current_distance, vertex = heappop(frontier)
        if current_distance > distances[vertex]:
            continue

        visit_order.append(vertex)

        for neighbour, weight in graph[vertex].items():
            if neighbour not in graph:
                raise ValueError(f"Neighbour '{neighbour}' referenced but not defined in graph.")
            new_distance = current_distance + weight
            if new_distance < distances[neighbour]:
                distances[neighbour] = new_distance
                predecessor[neighbour] = vertex
                heappush(frontier, (new_distance, neighbour))

    return {
        "distances": distances,
        "predecessor": predecessor,
        "visit_order": visit_order,
    }

In [16]:
result = dijkstra_shortest_paths(ASSIGNMENT_GRAPH, "C")

visit_order = result["visit_order"]
distances = result["distances"]

print("Visit order:", " -> ".join(visit_order))
print("\nShortest distances from C:")
for vertex in sorted(distances):
    value = distances[vertex]
    display_value = int(value) if value != INF else "Infinity"
    print(f"  {vertex}: {display_value}")

next_vertex = visit_order[1] if len(visit_order) > 1 else None
print(f"\nNext vertex visited after C: {next_vertex}")

Visit order: C -> B -> A -> E -> F -> G -> D

Shortest distances from C:
  A: 3
  B: 2
  C: 0
  D: 6
  E: 4
  F: 4
  G: 5

Next vertex visited after C: B


## Question 2
Using Prim's algorithm to build the minimum-cost network and suggest a cycle-forming edge.

In [13]:
from collections import defaultdict, deque
from heapq import heappush, heappop

INF = float("inf")


def prim_mst(num_vertices, edges, start=0):
    """Construct the MST using Prim's algorithm and capture auxiliary edge data."""
    if num_vertices <= 0:
        raise ValueError("Number of substations must be positive.")
    if not (0 <= start < num_vertices):
        raise ValueError("Start vertex must be within the valid range.")

    adjacency = defaultdict(list)
    edge_list = []

    for u, v, w in edges:
        if not (0 <= u < num_vertices and 0 <= v < num_vertices):
            raise ValueError("Edge references a substation outside the valid range.")
        if w < 0:
            raise ValueError("Cable installation costs must be non-negative.")
        adjacency[u].append((v, w))
        adjacency[v].append((u, w))
        edge_list.append((u, v, w))

    if len(adjacency) < num_vertices:
        raise ValueError("Graph must be connected; every substation needs at least one connection.")

    visited = {start}
    mst_edges = []
    total_cost = 0

    heap = []
    for neighbour, weight in adjacency[start]:
        heappush(heap, (weight, start, neighbour))

    while heap and len(visited) < num_vertices:
        weight, u, v = heappop(heap)
        if v in visited:
            continue
        visited.add(v)
        mst_edges.append((u, v, weight))
        total_cost += weight
        for next_vertex, next_weight in adjacency[v]:
            if next_vertex not in visited:
                heappush(heap, (next_weight, v, next_vertex))

    if len(visited) != num_vertices:
        raise ValueError("Input graph must be connected to form an MST.")

    return {
        "mst_edges": mst_edges,
        "total_cost": total_cost,
        "all_edges": edge_list,
    }


def _build_mst_adjacency(num_vertices, mst_edges):
    adjacency = {vertex: [] for vertex in range(num_vertices)}
    for u, v, w in mst_edges:
        adjacency[u].append((v, w))
        adjacency[v].append((u, w))
    return adjacency


def _max_edge_on_path(adjacency, start, end):
    queue = deque([start])
    parents = {start: (None, 0)}

    while queue and end not in parents:
        vertex = queue.popleft()
        for neighbour, weight in adjacency[vertex]:
            if neighbour not in parents:
                parents[neighbour] = (vertex, weight)
                queue.append(neighbour)

    if end not in parents:
        return None

    max_weight = 0
    cursor = end
    while parents[cursor][0] is not None:
        max_weight = max(max_weight, parents[cursor][1])
        cursor = parents[cursor][0]
    return max_weight


def smallest_cycle_edge(num_vertices, mst_edges, all_edges, mst_total_cost):
    """Identify the non-MST edge whose inclusion increases cost the least."""
    mst_edge_set = {frozenset((u, v)) for u, v, _ in mst_edges}
    mst_adj = _build_mst_adjacency(num_vertices, mst_edges)

    best_edge = None
    best_increase = INF

    for u, v, w in all_edges:
        if frozenset((u, v)) in mst_edge_set:
            continue
        max_on_path = _max_edge_on_path(mst_adj, u, v)
        if max_on_path is None:
            continue  # Graph is connected, so this should not happen
        increase = w - max_on_path
        if increase <= 0:
            continue
        if increase < best_increase:
            best_increase = increase
            best_edge = (u, v, w)

    if best_edge is None:
        return None

    return {
        "edge": best_edge,
        "increase": best_increase,
        "new_total_cost": mst_total_cost + best_increase,
    }

In [14]:
num_substations = 6
possible_connections = [
    (0, 1, 4),
    (0, 2, 3),
    (1, 2, 1),
    (1, 3, 2),
    (2, 3, 4),
    (3, 4, 2),
    (3, 5, 3),
    (4, 5, 6),
    (2, 5, 7),
]

mst_summary = prim_mst(num_substations, possible_connections, start=0)
mst_edges = mst_summary["mst_edges"]
total_cost = mst_summary["total_cost"]

extra_edge_info = smallest_cycle_edge(
    num_substations,
    mst_edges,
    mst_summary["all_edges"],
    total_cost,
)

print("Edges in the MST (u - v : cost):")
for u, v, w in mst_edges:
    print(f"  {u} - {v} : {w}")

print(f"\nTotal minimum installation cost: {total_cost}")

if extra_edge_info:
    extra_u, extra_v, extra_w = extra_edge_info["edge"]
    increase = extra_edge_info["increase"]
    new_total = extra_edge_info["new_total_cost"]
    print("\nCheapest additional edge forming a cycle:")
    print(f"  {extra_u} - {extra_v} : {extra_w}")
    print(f"  Cost increase: {int(increase)}")
    print(f"  New total if included: {int(new_total)}")
else:
    print("\nNo non-MST edge would increase the total cost (all alternatives are equal or cheaper).")

Edges in the MST (u - v : cost):
  0 - 2 : 3
  2 - 1 : 1
  1 - 3 : 2
  3 - 4 : 2
  3 - 5 : 3

Total minimum installation cost: 11

Cheapest additional edge forming a cycle:
  0 - 1 : 4
  Cost increase: 1
  New total if included: 12
